In [ ]:
import json
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)


In [ ]:
f = '/app/data/cloud_data_service/export-2019-03-14T22_56_48.469Z.json'
annotation_objects = json.load(open(f))

<h1> Prepare the data </h1>

In [ ]:
body_parts = []
for k, v in annotation_objects[0]['Label'].items():
    body_parts.append(k)
    



In [ ]:
annotation_objects[0]

In [ ]:
keypoints_df = pd.DataFrame()
fish_ids = []
s3_bucket = 'aquabyte-groundtruths'
for idx, obj in enumerate(annotation_objects):
    if obj['Label'] == 'Skip':
        continue
    
    # get fish_id
    image_url = str(obj['Labeled Data'])
    s3_bucket_key = image_url[image_url.index(s3_bucket):]
    s3_key = os.path.join(*s3_bucket_key.split('/')[1:])
    fish_id = image_url.split('/')[-3]
 
    # get image file name and epoch
    image_f_name = image_url.split('/')[-1]
    epoch = int(image_f_name.replace('.jpg', '').split('_')[-1])
    
    camera = str(image_f_name.split('_')[0])
    
    if fish_id not in fish_ids:
        fish_ids.append(fish_id)
    
    for body_part in body_parts:
        if body_part not in obj['Label'].keys():
            break
        kp_dict = obj['Label'][body_part][0]['geometry']
        kp = (kp_dict['x'], kp_dict['y'])
        
        row = {
            'fish_id': fish_id,
            'epoch': epoch,
            'image_s3_key': s3_key,
            's3_bucket': s3_bucket,
            'body_part': body_part,
            'camera': camera,
            'keypoint': kp
            
        }

        keypoints_df = keypoints_df.append(row, ignore_index=True)
        
        
        

In [ ]:
keypoints_df

<h1> Get 3D Distances </h1>

In [ ]:
# define some constants

FOCAL_LENGTH = 0.0084366
BASELINE = 0.128096
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000

In [ ]:
def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 4096)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 3000)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])



def depth_from_disp(disp):
    """ calculate the depth of the point based on the disparity value """
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth


def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5

In [ ]:
body_parts

In [ ]:
stereo_frame_pairs_df = pd.DataFrame()

for fish_id in fish_ids:
    fish_id_mask = keypoints_df.fish_id == fish_id
    epochs = keypoints_df[fish_id_mask].epoch.unique()
    for epoch in epochs:
        epoch_mask = keypoints_df.epoch == epoch
        row = {}
        row['gtsf_data_collection_id'] = 1
        row['gtsf_fish_identifier'] = fish_id
        row['epoch'] = epoch
        
        row['left_image_s3_key'] = keypoints_df[fish_id_mask \
                                                & epoch_mask \
                                                & (keypoints_df.camera == 'left')
                                               ].image_s3_key.iloc[0]
        
        row['right_image_s3_key'] = keypoints_df[fish_id_mask \
                                                & epoch_mask \
                                                & (keypoints_df.camera == 'right')
                                               ].image_s3_key.iloc[0]
        
        row['image_s3_bucket'] = keypoints_df[fish_id_mask].iloc[0].s3_bucket
            
        left_keypoints, right_keypoints, world_keypoints = {}, {}, {}
                    
        for body_part in body_parts:
            left_row = keypoints_df[
                (keypoints_df.epoch == epoch) & (keypoints_df.camera == 'left') & (keypoints_df.body_part == body_part)
            ].iloc[0]

            lkp = left_row['keypoint']
            left_keypoints[body_part] = lkp

            right_row = keypoints_df[
                (keypoints_df.epoch == epoch) & (keypoints_df.camera == 'right') & (keypoints_df.body_part == body_part)
            ].iloc[0]

            rkp = right_row['keypoint']
            right_keypoints[body_part] = rkp

            d = abs(lkp[0] - rkp[0])

            # compute world key point
            depth = depth_from_disp(d)
            wkp = convert_to_world_point(lkp[0], lkp[1], depth)

            world_keypoints[body_part] = wkp
            
        row['left_keypoints'] = left_keypoints
        row['right_keypoints'] = right_keypoints
        row['world_keypoints'] = world_keypoints
            
        stereo_frame_pairs_df = stereo_frame_pairs_df.append(row, ignore_index=True)



In [ ]:

stereo_frame_pairs_df[stereo_frame_pairs_df.gtsf_fish_identifier=='190226010001'].left_keypoints.iloc[0]

In [ ]:
annotations_by_datarow_id = {}
for obj in annotation_objects:
    datarow_id = obj['DataRow ID']
    if datarow_id not in list(annotations_by_datarow_id.keys()):
        annotations_by_datarow_id[datarow_id] = obj
    else:
        if obj['Created At'] > annotations_by_datarow_id[datarow_id]:
            annotations_by_datarow_id[datarow_id] = obj
    


In [ ]:
annotation_objects[0]['Project Name']